In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('info.csv')

In [3]:
#拆分数据集
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data['center'],data['steering'],test_size =0.2 ,random_state=25)
print("训练集{},测试集{}".format(len(X_train),len(X_test)))

C:\Users\Administrator\Anaconda3\envs\tfenv\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Administrator\Anaconda3\envs\tfenv\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


训练集1100,测试集276


C:\Users\Administrator\Anaconda3\envs\tfenv\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
#完整的数据增强器
from imgaug import augmenters as iaa
import numpy as np

def zoom_img(img):
    # 得到一个图像的缩放器
    zoom = iaa.Affine(scale=(1.0,1.3))
    return zoom.augment_image(img)
def trans_img(img):
    # 得到一个图像的平移器
    trans = iaa.Affine(translate_percent={'x':(-0.1,0.1),'y':(-0.1,0.1)})
    return trans.augment_image(img)
def img_random_brightness(img):
    #亮度变换的数据增强器
    brightness = iaa.Multiply((0.3,1.2))
    return brightness.augment_image(img)
def img_gaussian_blur(img):
    #高斯模糊的数据增强器
    gaussian_blur = iaa.GaussianBlur(sigma=(0.1,3))
    return gaussian_blur.augment_image(img)
def img_motion_blur(img):
    #高斯模糊的数据增强器
    motion_blur = iaa.MotionBlur()
    return motion_blur.augment_image(img)
def img_flip(img,steering_angle):
    #高斯模糊的数据增强器
    flip = iaa.Fliplr(1.0)
    return flip.augment_image(img),-steering_angle

## 自定义组合算法， 随机的数据增强。
## path，图片路径。 steering_angle方向盘角度
def random_augment(path,steering_angle):
    image = plt.imread(path)
    if np.random.rand()<0.5:
        image = zoom_img(image)
    if np.random.rand()<0.5:
        image = trans_img(image)
    if np.random.rand()<0.5:
        image = img_random_brightness(image)
    if np.random.rand()<0.5:
        image = img_gaussian_blur(image)
    if np.random.rand()<0.5:
        image = img_motion_blur(image)
    if np.random.rand()<0.5:
        image,steering_angle = img_flip(image,steering_angle)   
    return image,steering_angle

## 训练集数据增强器

In [5]:
import cv2
def train_batch_generator(batch_size):
    while True:
        batch_img = [] # 批量生成的图片
        batch_angle = [] # 每个图片对应一个角度
        for i in range(batch_size):
            ## 从原始的训练集里面随机获取一张照片
            index = np.random.randint(0,len(X_train))
            ## 获取训练集图片的路径
            path = X_train.iloc[index]
            ## 获取训练集图片对应的方向盘角度
            angle = float(y_train.iloc[index])         
            newimg, newangle = random_augment(path,angle)
            ## 对训练数据进行预处理
            newimg = newimg[60:135,:,:]
            newimg = cv2.GaussianBlur(newimg,(3,3),1)
            newimg = cv2.resize(newimg,(200,66))
            newimg = cv2.cvtColor(newimg,cv2.COLOR_RGB2YUV)
            newimg = newimg/255.0
            
            batch_img.append(newimg)
            batch_angle.append(newangle)
        yield np.array(batch_img),np.array(batch_angle)

## 测试集的数据增强器

In [6]:
import cv2
def test_batch_generator(batch_size):
    while True:
        batch_img = [] # 批量生成的图片
        batch_angle = [] # 每个图片对应一个角度
        for i in range(batch_size):
            ## 从原始的训练集里面随机获取一张照片
            index = np.random.randint(0,len(X_test))
            ## 获取训练集图片的路径
            path = X_test.iloc[index]
            ## 获取训练集图片对应的方向盘角度
            angle = float(y_test.iloc[index])         
            newimg, newangle = random_augment(path,angle)
            ## 对训练数据进行预处理
            newimg = newimg[60:135,:,:]
            newimg = cv2.GaussianBlur(newimg,(3,3),1)
            newimg = cv2.resize(newimg,(200,66))
            newimg = cv2.cvtColor(newimg,cv2.COLOR_RGB2YUV)
            newimg = newimg/255.0
            
            batch_img.append(newimg)
            batch_angle.append(newangle)
        yield np.array(batch_img),np.array(batch_angle)

## 定义模型

In [7]:
import tensorflow as tf 
# 创建一个顺序模型
model = tf.keras.Sequential()
# normalnation后的第一层
model.add(tf.keras.layers.Conv2D(24,(5,5),strides=(2,2),input_shape=(66,200,3),activation='elu'))
# 第二层卷积
model.add(tf.keras.layers.Conv2D(36,(5,5),strides=(2,2),activation='elu'))
# 第三层卷积
model.add(tf.keras.layers.Conv2D(48,(5,5),strides=(2,2),activation='elu'))
# 第四层卷积
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='elu'))
# 第五层卷积
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='elu'))

# 防止训练过拟合
model.add(tf.keras.layers.Dropout(0.5))

# 全连接打平
model.add(tf.keras.layers.Flatten())

# 全连接的隐藏层
model.add(tf.keras.layers.Dense(1164,activation='elu'))
# 防止训练过拟合
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(100,activation='elu'))
# 防止训练过拟合
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(50,activation='elu'))
# 防止训练过拟合
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(10,activation='elu'))

model.add(tf.keras.layers.Dense(1)) # 方向盘的角度。

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
dropout (Dropout)            (None, 1, 18, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [10]:
## 编译模型,回归问题
model.compile(optimizer='adam',loss='mse')

## 使用数据增强器训练的话，api有细微差别
model.fit_generator(train_batch_generator(1000),
                    steps_per_epoch=10,
                    epochs=10,
                   validation_data=test_batch_generator(300),
                    validation_steps=10
                   )

Epoch 1/10
10/10 [==============================] - 79s 8s/step - loss: 2.7369 - val_loss: 0.3166
Epoch 2/10
10/10 [==============================] - 78s 8s/step - loss: 0.7975 - val_loss: 0.1974
Epoch 3/10
10/10 [==============================] - 78s 8s/step - loss: 0.4167 - val_loss: 0.1062
Epoch 4/10
10/10 [==============================] - 79s 8s/step - loss: 0.3013 - val_loss: 0.1665
Epoch 5/10
10/10 [==============================] - 78s 8s/step - loss: 0.2392 - val_loss: 0.1187
Epoch 6/10
10/10 [==============================] - 78s 8s/step - loss: 0.4005 - val_loss: 0.3439
Epoch 7/10
10/10 [==============================] - 78s 8s/step - loss: 0.3524 - val_loss: 0.1358
Epoch 8/10
10/10 [==============================] - 79s 8s/step - loss: 0.1944 - val_loss: 0.1164
Epoch 9/10
10/10 [==============================] - 79s 8s/step - loss: 0.1919 - val_loss: 0.1206
Epoch 10/10
10/10 [==============================] - 79s 8s/step - loss: 0.1453 - val_loss: 0.1147


In [15]:
## 加载上一次课的验证集的数据文件
npzfile = np.load('X_test.npz')
last_X_test = npzfile['arr_0']
npzfile = np.load('y_test.npz')
last_y_test = npzfile['arr_0']
model.evaluate(last_X_test,last_y_test)

9/9 [==============================] - 0s 22ms/step - loss: 0.1117


0.11174007505178452

In [16]:
model.save('2.h5')

## 训练的过程中，保存每次训练的结果

In [9]:
# 使用一个特殊的回调，动态保存每次训练的结果
from tensorflow.keras.callbacks import ModelCheckpoint

## 编译模型,回归问题
model.compile(optimizer='adam',loss='mse')
checkpoint = ModelCheckpoint('{epoch:04d}.h5')
## 使用数据增强器训练的话，api有细微差别
model.fit_generator(train_batch_generator(100),
                    steps_per_epoch=10,
                    epochs=10,
                    validation_data=test_batch_generator(30),
                    validation_steps=10,
                    callbacks=[checkpoint]
                   )

Epoch 1/10
10/10 [==============================] - 8s 826ms/step - loss: 5.7208 - val_loss: 0.8426
Epoch 2/10
10/10 [==============================] - 9s 873ms/step - loss: 1.0882 - val_loss: 0.1336
Epoch 3/10
10/10 [==============================] - 8s 812ms/step - loss: 0.4498 - val_loss: 0.1246
Epoch 4/10
10/10 [==============================] - 9s 861ms/step - loss: 0.2809 - val_loss: 0.1177
Epoch 5/10
10/10 [==============================] - 9s 852ms/step - loss: 0.1757 - val_loss: 0.1478
Epoch 6/10
10/10 [==============================] - 8s 848ms/step - loss: 0.1403 - val_loss: 0.1428
Epoch 7/10
10/10 [==============================] - 8s 832ms/step - loss: 0.1454 - val_loss: 0.1182
Epoch 8/10
10/10 [==============================] - 8s 840ms/step - loss: 0.1206 - val_loss: 0.1206
Epoch 9/10
10/10 [==============================] - 8s 807ms/step - loss: 0.1262 - val_loss: 0.1080
Epoch 10/10
10/10 [==============================] - 7s 748ms/step - loss: 0.1237 - val_loss: 0.1051

## earlystopping技术

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
# 创建一个动态早停止技术
# monitor监控哪个指标
# patience 耐心指数
# restore_best_weights 重新加载最好的权重
earlystopping = EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)
model.compile(optimizer='adam',loss='mse')
model.fit_generator(train_batch_generator(100),
                    steps_per_epoch=10,
                    epochs=100,
                    validation_data=test_batch_generator(30),
                    validation_steps=10,
                    callbacks=[earlystopping]
                   )

Epoch 1/100
10/10 [==============================] - 8s 848ms/step - loss: 2.9253 - val_loss: 0.2114
Epoch 2/100
10/10 [==============================] - 8s 802ms/step - loss: 0.4338 - val_loss: 0.2805
Epoch 3/100
10/10 [==============================] - 8s 817ms/step - loss: 0.4100 - val_loss: 0.1206
Epoch 4/100
10/10 [==============================] - 8s 837ms/step - loss: 0.3757 - val_loss: 0.1373
Epoch 5/100
10/10 [==============================] - 8s 812ms/step - loss: 0.2378 - val_loss: 0.1210
Epoch 6/100
10/10 [==============================] - 8s 803ms/step - loss: 0.2014 - val_loss: 0.0926
Epoch 7/100
10/10 [==============================] - 8s 802ms/step - loss: 0.1453 - val_loss: 0.1094
Epoch 8/100
10/10 [==============================] - 8s 803ms/step - loss: 0.1400 - val_loss: 0.1096
Epoch 9/100
10/10 [==============================] - 8s 806ms/step - loss: 0.1273 - val_loss: 0.1072
Epoch 10/100
10/10 [==============================] - 8s 799ms/step - loss: 0.1194 - val_lo

## 动态学习速率技术

In [11]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=3,verbose=1,min_delta=0.0001)
model.compile(optimizer='adam',loss='mse')
model.fit_generator(train_batch_generator(100),
                    steps_per_epoch=10,
                    epochs=30,
                    validation_data=test_batch_generator(30),
                    validation_steps=10,
                    callbacks=[reduce_lr]
                   )

Epoch 1/30
10/10 [==============================] - 8s 810ms/step - loss: 0.7387 - val_loss: 0.3353
Epoch 2/30
10/10 [==============================] - 8s 802ms/step - loss: 0.3682 - val_loss: 0.2718
Epoch 3/30
10/10 [==============================] - 8s 792ms/step - loss: 0.2647 - val_loss: 0.1266
Epoch 4/30
10/10 [==============================] - 8s 810ms/step - loss: 0.3143 - val_loss: 0.2720
Epoch 5/30
10/10 [==============================] - 8s 810ms/step - loss: 0.2901 - val_loss: 0.1745
Epoch 6/30
10/10 [==============================] - ETA: 0s - loss: 0.2278
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
10/10 [==============================] - 8s 817ms/step - loss: 0.2278 - val_loss: 0.1410
Epoch 7/30
10/10 [==============================] - 8s 844ms/step - loss: 0.1522 - val_loss: 0.1054
Epoch 8/30
10/10 [==============================] - 8s 846ms/step - loss: 0.1447 - val_loss: 0.1063
Epoch 9/30
10/10 [==============================] - 8s 